In [1]:


import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random
import os


os.makedirs("graficos", exist_ok=True)


dados = pd.read_csv("comentarios.csv")


dados = dados.dropna(subset=['comentario'])


X = dados['comentario']


sentimentos_demo = ['Positivo', 'Negativo', 'Neutro']
dados['sentimento_real'] = [random.choice(sentimentos_demo) for _ in range(len(dados))]


vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(X)
modelo = MultinomialNB()
modelo.fit(X_vect, dados['sentimento_real'])


dados['sentimento_predito'] = modelo.predict(X_vect)


print(dados.head())


df_plot = dados['sentimento_predito'].value_counts().reset_index()
df_plot.columns = ['sentimento', 'quantidade']

fig = px.bar(
    df_plot,
    x='sentimento',
    y='quantidade',
    color='sentimento',
    color_discrete_map={'Positivo':'green','Negativo':'red','Neutro':'blue'},
    title='Distribuição de Sentimentos dos Comentários'
)


fig.write_html("graficos/grafico_sentimentos.html")
print("Dashboard salvo em: graficos/grafico_sentimentos.html")


for sentimento in sentimentos_demo:
    texto_sentimento = " ".join(dados[dados['sentimento_predito']==sentimento]['comentario'])
    if texto_sentimento.strip(): 
        wc = WordCloud(width=800, height=400, background_color="white").generate(texto_sentimento)
        plt.figure(figsize=(10,5))
        plt.imshow(wc, interpolation="bilinear")
        plt.axis("off")
        plt.title(f"Nuvem de palavras - {sentimento}")
     
        filename = f"graficos/wordcloud_{sentimento}.png"
        plt.savefig(filename)
        plt.close()
        print(f"WordCloud salva em: {filename}")


                                comentario sentimento_real sentimento_predito
Adorei o produto          super recomendo!          Neutro             Neutro
Péssimo atendimento        não volto mais.        Positivo           Positivo
Entrega no prazo               tudo certo.        Negativo           Negativo
Muito bom             qualidade excelente!          Neutro             Neutro
Não gostei            demorou para chegar.        Positivo           Positivo
Dashboard salvo em: graficos/grafico_sentimentos.html
WordCloud salva em: graficos/wordcloud_Positivo.png
WordCloud salva em: graficos/wordcloud_Negativo.png
WordCloud salva em: graficos/wordcloud_Neutro.png
